In [77]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time

In [87]:
# scrape from 2018 - 2024
for yr in range(2018,2025) :

    URL = 'https://www.tennesseetitans.com/schedule/' + str(yr) +'/'
    
    response = requests.get(URL)
    
    soup = BS(response.text)
    
    df = pd.DataFrame(columns = ['season','date','outcome','score','prefix','vs'])
    
    season = 'Regular Season'

    # loop through the games listed on the site, and add each one to the dataframe
    i = 0
    for game in soup.findAll('div', attrs={'class' : 'nfl-o-matchup-cards nfl-o-matchup-cards--post-game'}) :
        if i > 0 and game['data-gametime'][0:10] < df.loc[i-1,'date'] :
            season = 'Postseason'
        df.loc[i,'season'] = season
        df.loc[i,'date'] = game['data-gametime'][0:10]
        df.loc[i,'outcome'] = game.find('span', attrs={'class' : 'nfl-o-matchup-cards__score--result'}).text
        df.loc[i,'score'] = game.find('span', attrs={'class' : 'nfl-o-matchup-cards__score--points'}).text
        if game.find('span', attrs={'class' : 'nfl-o-matchup-cards__team-prefix'}) != None :
            df.loc[i,'prefix'] = game.find('span', attrs={'class' : 'nfl-o-matchup-cards__team-prefix'}).text
        else :
            df.loc[i,'prefix'] = ''
        df.loc[i,'vs'] = game.find('p', attrs={'class' : 'nfl-o-matchup-cards__team-full-name'}).text.strip()
        i += 1

    # save each file as a csv, and wait 3 seconds to not ping too frequently
    df.to_csv('../data/titans_games/titans_' + str(yr) + '.csv', index=False)
    time.sleep(3)